In [1]:
import os
os.chdir('../src/models/')
import sys
from dotenv import load_dotenv, find_dotenv
import numpy as np

sys.path.append(os.path.abspath("../.."))
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()
# load up the entries as environment variables
load_dotenv(dotenv_path)
# import pandas as pd

from src.code_snippets.dataprep.embeddings_preprocessing.data_preparation import (
    sentences_to_indices,
    pretrained_embedding_layer
)

from src.code_snippets.utils.abstract_classes import Trainer
from src.code_snippets.dataprep.embeddings_preprocessing.glove.reader import read_glove_file,get_word_index_dicts
from src.code_snippets.dataprep.embeddings_preprocessing.data_preparation import pretrained_embedding_layer
import random


In [2]:

class ManyToOneSeqModel(Trainer):

    def __init__(self,train_data,val_data,embedding_dir):
        self.train_data = train_data
        self.val_data = val_data
        self.embedding_dir = embedding_dir
        self.gensim_model = read_glove_file(self.embedding_dir)
        self.word_to_index,self.index_to_words = get_word_index_dicts(self.gensim_model)
        
        self.m_X, self.n_X = self.train_data['X_indices_train'].shape
        self.m_X_aux, self.n_X_aux = self.train_data['X_aux_train'].shape
    
    def set_model(self):
        pretrained_embedding_layer(self.gensim_model, self.word_to_index)
    
    def save_model(self):
        pass
    def fit_model(self):
        pass
    def generate_metrics(self):
        pass

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input,LSTM,Conv1D,Dropout, Dense
from tensorflow.keras import Model
import tensorflow_addons as tfa

In [24]:
seed = 100
random.seed(seed)
np.random.seed(seed)

In [6]:
train_data = {
              'X_aux_train': np.load('../../data/processed/X_aux_train.npy'),
              'X_indices_train':np.load('../../data/processed/X_indices_train.npy').astype('int32'),
              'y_train' : np.load('../../data/processed/y_train.npy')
             }

val_data = {
            'X_aux_val': np.load('../../data/processed/X_aux_val.npy'),
            'X_indices_val': np.load('../../data/processed/X_indices_val.npy').astype('int32'),
            'y_val' : np.load('../../data/processed/y_val.npy')
           }

In [30]:
num_observations = len(train_data['X_aux_train'])
idx = np.arange(num_observations)

sample_portion = 0.01
sample_size = int(num_observations*sample_portion)
print(sample_size)

1531


In [42]:
np.random.seed(seed)
sampled_idx = np.random.choice(idx,sample_size,replace = False)

train_data['X_indices_train'] = train_data['X_indices_train'][sampled_idx]
train_data['X_aux_train'] = train_data['X_aux_train'][sampled_idx]
train_data['y_train'] = train_data['y_train'][sampled_idx]

In [44]:
trainer = ManyToOneSeqModel(train_data,
                            val_data,
                            "../../../../pretrained_embeddings/glove.twitter.27B/glove.twitter.27B.25d.txt")


In [53]:
#TODO: uncomment this below
#sentence_indices = Input((trainer.n_X),dtype='int32')
sentence_indices = Input((100),dtype='int32')


# Create the embedding layer pretrained with GloVe Vectors (≈1 line)
embedding_layer = pretrained_embedding_layer(trainer.gensim_model,
                                             trainer.word_to_index)

# Propagate sentence_indices through your embedding layer
# (See additional hints in the instructions).
embeddings = embedding_layer(sentence_indices)   

X = LSTM(128,return_sequences=True)(embeddings)

X = Dropout(0.1,seed= seed)(X)

X = LSTM(128,return_sequences=False)(X)

X = Dropout(0.1,seed= seed)(X)

X = Dense(16,activation='relu')(X)

X = Dropout(0.1,seed= seed)(X)

X = Dense(1,activation='sigmoid')(X)


model = Model(sentence_indices,X)

In [54]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 25)           29837875  
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 128)          78848     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 128)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                2064

In [47]:
import tensorflow.keras.backend as K

def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val


In [56]:
model.compile(loss='binary_crossentropy',metrics = [f1_metric])


In [58]:
model.fit(trainer.train_data['X_indices_train'][:,0:100],
          trainer.train_data['y_train'],
          epochs=50,
          batch_size=2**5,
          shuffle=True)

Train on 1531 samples
Epoch 1/50
1531/1531 [==============================] - 13s 9ms/sample - loss: 0.3509 - f1_metric: 0.0409
Epoch 2/50
1531/1531 [==============================] - 9s 6ms/sample - loss: 0.2919 - f1_metric: 0.2399
Epoch 3/50
1376/1531 [=========================>....] - ETA: 0s - loss: 0.2697 - f1_metric: 0.3746

KeyboardInterrupt: 

In [49]:
model.fit(trainer.train_data['X_indices_train'][:,0:100],
          trainer.train_data['y_train'][:,0:100],
          epochs=50,
          batch_size=2**5,
          shuffle=True)

Train on 1531 samples
Epoch 1/50
1531/1531 [==============================] - 266s 174ms/sample - loss: 0.3504 - f1_metric: 0.0372
Epoch 2/50
1531/1531 [==============================] - 276s 181ms/sample - loss: 0.2870 - f1_metric: 0.2918
Epoch 3/50
1531/1531 [==============================] - 262s 171ms/sample - loss: 0.2603 - f1_metric: 0.4333
Epoch 4/50
1531/1531 [==============================] - 286s 187ms/sample - loss: 0.2319 - f1_metric: 0.4627
Epoch 5/50
1531/1531 [==============================] - 324s 212ms/sample - loss: 0.2250 - f1_metric: 0.4811
Epoch 6/50
1531/1531 [==============================] - 345s 225ms/sample - loss: 0.2103 - f1_metric: 0.5469
Epoch 7/50
1531/1531 [==============================] - 362s 237ms/sample - loss: 0.2050 - f1_metric: 0.5221
Epoch 8/50
1531/1531 [==============================] - 354s 232ms/sample - loss: 0.1884 - f1_metric: 0.6368
Epoch 9/50
1531/1531 [==============================] - 289s 189ms/sample - loss: 0.1758 - f1_metric: 0.59

KeyboardInterrupt: 